In [1]:
"""
This notebook is used to perform RFECV and assemble the resulting feature lists (and those from feature_selection_test.ipynb) into a dictionary to be pickled and used later.

It also provides basic hyperparameter search spaces for our classifiers. Spaces here will be updated as we do more work and find better tuning ranges.

Author: Wesley
"""
import pandas as pd
import numpy as np
from skopt.space import Categorical, Integer, Real

from sklearn.feature_selection import RFECV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import LinearSVC

import pickle

from sklearn.multiclass import OneVsRestClassifier

import traceback

In [2]:
binary = pd.read_csv("binary.csv")
multiclass = pd.read_csv("multiclass.csv")

We perform RFE using RFECV for all classifiers for which this is possible. Binary is first.

In [3]:
# Encode binary labels and prepare X and y
binary[" Label"] = binary[" Label"].apply(lambda x: 0 if x == "BENIGN" else 1)
y = binary[" Label"].copy()
X = binary.drop([" Label"], axis=1)

In [8]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42),
    "Linear SVC": make_pipeline(StandardScaler(), LinearSVC(random_state=42)),
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
}

# This is a suitable scoring method because the classes are balanced.
score_method = 'accuracy'

for name, model in models.items():
    if name == "Linear SVC":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.linearsvc.coef_",
        )

    elif name == "Logistic Regression":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.logisticregression.coef_",
        )

    else:            
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
        )

    try:      
        rfecv.fit(X, y)
        features = X.columns[rfecv.support_].tolist()
        print(f"{name}: {features}")

    except:
        print(f"{name} failed to generate features.\n")

Decision Tree: [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' UR

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  

Linear SVC: [' Protocol', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow IAT Max', ' Fwd IAT Max', ' Fwd IAT Min', 'Fwd PSH Flags', ' Fwd Header Length', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' Average Packet Size', ' Avg Bwd Segment Size', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' Active Min', ' Idle Max', ' Inbound']
Logistic Regression failed to generate features.



c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Multiclass is next. LinearSVC was abandoned due to being too slow

In [4]:
le = LabelEncoder()
multiclass[" Label"] = le.fit_transform(multiclass[" Label"].values)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(multiclass[' Label']))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)


Multiclass Label Encodings (in order of digits 0 -> n): 
['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


In [5]:
multiclass_y = multiclass[" Label"].copy()
multiclass_X = multiclass.drop([" Label"], axis=1)

In [17]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42, num_class=11, objective='multi:softmax'),
    "Linear SVC": make_pipeline(StandardScaler(), LinearSVC(random_state=42)),
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
}

# This is a suitable scoring method because the classes are balanced.
score_method = 'accuracy'

for name, model in models.items():
    if name == "Linear SVC":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.linearsvc.coef_",
        )

    elif name == "Logistic Regression":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.logisticregression.coef_",
        )

    else:            
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
        )

    try:      
        rfecv.fit(multiclass_X, multiclass_y)
        features = multiclass_X.columns[rfecv.support_].tolist()
        print(f"{name}: {features}")

    except:
        traceback.print_stack()
        print(f"{name} failed to generate features.\n")

Decision Tree: [' Flow Duration', ' Total Fwd Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' SYN Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', ' Inbound']
Random

  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\asyncio\

XGBoost failed to generate features.



Assembled search spaces and feature lists.

In [6]:
# Hyperparameter search spaces for eligible classifiers. Currently set to generic ranges, will be updates as we test them.
search_spaces = {
    "XGBoost": {
            "max_depth": Integer(1, 30),
            "gamma": Real(0.1, 10, prior="log-uniform"),
            "subsample": Real(0.1, 1, prior="log-uniform"),
            "min_child_weight": Integer(1, 10),
            "colsample_bytree": Real(0.1, 1, prior="log-uniform"),
            "learning_rate": Real(0.01, 1, prior="log-uniform"),
            "max_delta_step": Integer(0, 10),
            "lambda": Integer(0, 3),
            "alpha": Integer(0, 2),
        },
    "Bagging SVM": {
            "warm_start": Categorical([True, False]),
            "bootstrap": Categorical([True, False]),
            "n_estimators": Integer(5, 50),   
        },
    "SVC (RBF)": {
            "svc__C": Real(0.00001, 10000, prior="log-uniform"),
            "svc__gamma": Real(0.000001, 100, prior="log-uniform"),
         
    },
    "SVC (Poly)": {
            "svc__C": Real(0.00001, 10000, prior="log-uniform"),
            "svc__gamma": Real(0.000001, 100, prior="log-uniform"),
            "svc__degree": Integer(1,20),
            "svc__coef0": Categorical([0,1]),
    },   
    "Logistic Regression": {       
            "logisticregression__C": Real(0.00001, 10000, prior="log-uniform"),
            "logisticregression__max_iter": Integer(100, 500),
    },
    "Random Forest": {
            "criterion": Categorical(["gini", "entropy"]),
            "max_depth": Integer(3, 100),
            "max_features": Categorical(["sqrt", "log2"]),
            "min_samples_leaf": Integer(1, 20),
            "min_samples_split": Integer(2, 20),
            "n_estimators": Integer(50, 500),
    },
    "KNN": {
            "n_neighbors": Integer(2,50),
            "weights": Categorical(["uniform","distance"]),
            "algorithm": Categorical(["ball_tree","kd_tree", "brute"]),
    },
    "Linear SVC": {
            "linearsvc__C": Real(0.0001, 10000, prior="log-uniform"),
            "linearsvc__max_iter": Integer(1000, 10000)
    },  
    "Naive Bayes": {
        "var_smoothing": Real(1e-9,1e-7, prior="log-uniform")
    },
    "Decision Tree": {
            "criterion": Categorical(["gini", "entropy"]),
            "max_depth": Integer(3, 50),
            "max_features": Categorical(["sqrt", "log2"]),
            "min_samples_leaf": Integer(1, 10),
            "min_samples_split": Integer(2, 10),
    },
}

# Contains all currently chosen feature sets from RFE, MI, and Pearson Correlation-Based Feature Selection. Also contains chosen variance threshold for PCA.
feature_lists = {
    "Binary": {
        "All": X.columns,
        "Correlation" : [' Protocol', ' Flow Duration', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', 'Init_Win_bytes_forward', ' Inbound'],
        "Mutual Information": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' URG Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', ' Inbound'],
        "RFE Sets": {
                "Decision Tree": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', 'SimillarHTTP', ' Inbound'],
                "Random Forest": [' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Flow Bytes/s', ' Bwd Header Length', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' ACK Flag Count', ' URG Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Subflow Bwd Packets', 'Init_Win_bytes_forward', ' min_seg_size_forward', ' Inbound'],
                "XGBoost": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Std', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', ' Bwd IAT Mean', ' Bwd IAT Std', 'Fwd PSH Flags', ' Fwd Header Length', 'Fwd Packets/s', ' Min Packet Length', ' Packet Length Std', ' SYN Flag Count', ' URG Flag Count', ' Average Packet Size', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' min_seg_size_forward', 'Active Mean', ' Inbound'],
                "Linear SVC": [' Protocol', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow IAT Max', ' Fwd IAT Max', ' Fwd IAT Min', 'Fwd PSH Flags', ' Fwd Header Length', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' Average Packet Size', ' Avg Bwd Segment Size', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' Active Min', ' Idle Max', ' Inbound'],
                "Logistic Regression": [' Protocol', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Max', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', 'Active Mean', ' Idle Max', ' Inbound'],
        },
        "PCA": 0.95,
    },
    "Multiclass": {
        "All": multiclass_X.columns,
        "Correlation": [' Protocol', ' Flow Duration', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Std', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', 'Bwd IAT Total', ' Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', 'Idle Mean', ' Idle Max', ' Idle Min', ' Inbound',' Fwd Header Length', ' Fwd Header Length.1', ' min_seg_size_forward', ' Total Fwd Packets', 'Subflow Fwd Packets', ' act_data_pkt_fwd'],
        "Mutual Information": [' Protocol', ' Flow Duration', ' Total Fwd Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', ' Fwd Header Length', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', ' min_seg_size_forward'],
        "RFE Sets": {
                "Decision Tree": [' Flow Duration', ' Total Fwd Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' SYN Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', ' Inbound'],
                "Random Forest": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', ' Inbound'],
                "XGBoost": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' SYN Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', ' Inbound'],
                "Logistic Regression": [' Protocol', ' Flow Duration', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', ' Bwd Packet Length Mean', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min']
        },
        "PCA": 0.95,
    }
}

Dump results for use later.

In [7]:
# Save dictionaries
pickle.dump(feature_lists, open("feature_sets.pickle", "wb"))
pickle.dump(search_spaces, open("hyperparameter_search_spaces.pickle", "wb"))